# Clase de scraping de Fotocasa.es
El siguiente código permite scrapear las propiedades de Fotocasa.es de diferentes maneras. Utilizando las librerías Selenium y BeautifulSoup para obtener la data de la propia web y concurrent.futures para acelerar la obtención, podemos conseguir diferentes datasets para la ciudad y el rango de páginas que especifiquemos como parámetros.

De igual modo, el script contempla tres modos de uso, uno que guarda el dato en un dataframe directamente, otro que lo guarda y además descarga cada propiedad en un archivo de texto, y un tercero que nos permite descargar o no y crear el dataframe desde la descarga realizada en el momento o previamente; este último modo nos permite trabajar con archivos en nuestro disco duro sin necesidad de volver a recorrer todas las páginas de la web.

La clase finalmente guarda el dataframe resultante en un archivo .csv por si queremos utilizar el dato más adelante.

In [295]:
%%time

from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--enable-javascript')
chrome_options.add_argument('user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36')

from bs4 import BeautifulSoup
import concurrent.futures
import os
import shutil
import numpy as np
import requests
import re
import time
import pandas as pd
#from lxml import etree, html
import json


class fotocasa_scraping:
  '''
  Clase para inicializar el scraping de Fotocasa.com
  '''

  def check_features(self, data, page):
    '''
    Comprobaciones de features para lectura correcta
    '''    
    # Diccionario de features
    realestate = {
        'origin': 'fotocasa',
        'page': page,
        'title': '',
        'link': '',
        'image_url': '',
        'country': '',
        'district': '',
        'neighborhood': '',
        'street': '',
        'zipCode': '',
        'province': '',
        'buildingType': '',
        'clientAlias': '',
        'latitude': '',
        'longitude': '',
        'isNewConstruction': '',
        'rooms': '',
        'bathrooms': '',
        'parking': '',
        'elevator': '',
        'furnished': '',
        'surface': '',
        'energyCertificate': '',
        'hotWater': '',
        'heating': '',
        'conservationState': '',
        'antiquity': '',
        'floor': '',
        'surfaceLand': '',
        'otherFeatures': '',
        'price': '',     
        }
    # Comienzan las comprobaciones feature a feature
    try:
        realestate['title'] = data['propertyTitle']    
    except:
        realestate['title'] = '^'
        
    try:
        realestate['link'] = 'https://www.fotocasa.es' + data['realEstate']['detail']['es-ES']   
    except:
        realestate['link'] = '^'

    try:
        realestate['image_url'] = data['realEstate']['multimedia'][1]['src']
    except:
        realestate['image_url'] = '^'
        
    try:
        realestate['country'] = data['realEstate']['address']['country']
    except:
        realestate['country'] = '^'
        
    try:
        realestate['district'] = data['realEstate']['address']['district']
    except:
        realestate['district'] = '^'
        
    try:
        realestate['neighborhood'] = data['realEstate']['address']['neighborhood']
    except:
        realestate['neighborhood'] = '^'
        
    try:
        realestate['street'] = data['realEstate']['location']
    except:
        realestate['street'] = '^'
        
    try:
        realestate['zipCode'] = data['realEstate']['address']['zipCode']
    except:
        realestate['zipCode'] = '^'
        
    try:
        realestate['province'] = data['realEstate']['address']['province']
    except:
        realestate['province'] = '^'
        
    try:
        realestate['buildingType'] = data['realEstate']['buildingType']
    except:
        realestate['buildingType'] = '^'

    try:
        realestate['clientAlias'] = data['realEstate']['clientAlias']
    except:
        realestate['clientAlias'] = '^'
        
    try:
        realestate['latitude'] = data['realEstate']['coordinates']['latitude']
    except:
        realestate['latitude'] = '^'

    try:
        realestate['longitude'] = data['realEstate']['coordinates']['longitude']
    except:
        realestate['longitude'] = '^'
        
    try:
        realestate['isNewConstruction'] = data['realEstate']['isNewConstruction']
    except:
        realestate['isNewConstruction'] = '^'
        
    try:
        realestate['rooms'] = data['realEstate']['features']['rooms']
    except:
        realestate['rooms'] = '^'
        
    try:
        realestate['bathrooms'] = data['realEstate']['features']['bathrooms']
    except:
        realestate['bathrooms'] = '^'

    try:
        featureList = data['realEstate']['featuresList']
        realestate['parking'] = ''.join([featureList[index]['value'] for index,value in enumerate(featureList) if featureList[index]['label'] == 'parking'])
        
    except:
        realestate['parking'] = '^'

    try:
        featureList = data['realEstate']['featuresList']
        realestate['elevator'] = ''.join([featureList[index]['value'] for index,value in enumerate(featureList) if featureList[index]['label'] == 'elevator'])
        
    except:
        realestate['elevator'] = '^'

    try:
        featureList = data['realEstate']['featuresList']
        realestate['furnished'] = ''.join([featureList[index]['value'] for index,value in enumerate(featureList) if featureList[index]['label'] == 'furnished'])
        
    except:
        realestate['furnished'] = '^'
        
    try:
        realestate['surface'] = data['realEstate']['features']['surface']
    except:
        realestate['surface'] = '^'
        
    try:
        realestate['energyCertificate'] = data['realEstate']['energyCertificate']
    except:
        realestate['energyCertificate'] = '^'
        
    try:
        realestate['hotWater'] = data['realEstate']['features']['hotWater']
        featureList = data['realEstate']['featuresList']
        realestate['hotWater'] = ''.join([featureList[index]['value'] for index,value in enumerate(featureList) if featureList[index]['label'] == 'hotWater'])
        
    except:
        realestate['hotWater'] = '^'
        
    try:
        realestate['heating'] = data['realEstate']['features']['heating']
        featureList = data['realEstate']['featuresList']
        realestate['heating'] = ''.join([featureList[index]['value'] for index,value in enumerate(featureList) if featureList[index]['label'] == 'heating'])
       
    except:
        realestate['heating'] = '^'
        
    try:
        realestate['conservationState'] = data['realEstate']['features']['conservationState']
        featureList = data['realEstate']['featuresList']
        realestate['conservationState'] = ''.join([featureList[index]['value'] for index,value in enumerate(featureList) if featureList[index]['label'] == 'conservationState'])
       
    except:
        realestate['conservationState'] = '^'
        
    try:
        realestate['antiquity'] = data['realEstate']['features']['antiquity']
        featureList = data['realEstate']['featuresList']
        realestate['antiquity'] = ''.join([featureList[index]['value'] for index,value in enumerate(featureList) if featureList[index]['label'] == 'antiquity'])
       
    except:
        realestate['antiquity'] = '^'
        
    try:
        realestate['floor'] = data['realEstate']['features']['floor']
    except:
        realestate['floor'] = '^'
        
    try:
        realestate['surfaceLand'] = data['realEstate']['features']['surfaceLand']
    except:
        realestate['surfaceLand'] = '^'
        
    try:
        realestate['otherFeatures'] = data['realEstate']['otherFeatures']
    except:
        realestate['otherFeatures'] = '^'
        
    try:
        realestate['price'] = data['realEstate']['price']
    except:
        realestate['price'] = 0
        
    #devuelve un diccionario
    return realestate

  def parse_properties(self, driver, url_list, page, download, files_mode):
    # Recibo una lista de urls de la propiedad y lo separo en un diccionario. Devuelvo dataframe.
    df_page = pd.DataFrame()
        
    i = 0
    #print(url_list)
    
    if download == True:
        directory = f'fotocasa/fotocasa_{page}'
        os.mkdir(directory)
    
    for url in url_list:
        driver.get(url)
        #element_present = EC.presence_of_element_located((By.XPATH, '//div[@id="modal-react-portal"]'))
        #WebDriverWait(driver, 5).until(element_present) # el driver debe esperarse a que la página se cargue        
        
        html_txt = driver.page_source
        soup = BeautifulSoup(html_txt,'html.parser')
        prop_scripts = soup.findAll('script')
        prop_features = ''.join([re.search('window.__INITIAL_PROPS__ = JSON.parse(.*)\n',str(x)).group(1) for x in prop_scripts if re.search('window.__INITIAL_PROPS__',str(x))])
        prop_features_clean = re.sub(r'\\"','"',prop_features)
        prop_features_clean = re.sub(r'\\\\"','',prop_features_clean)
        prop_features_clean = re.sub(r'\("|"\);','',prop_features_clean)
        prop_features_clean = re.sub(r',"seo":.*','}',prop_features_clean)

        try:
            prop_data = json.loads(prop_features_clean)
            realestate = self.check_features(prop_data,page)
            
            if download == True:
                self.download_realestates(prop_features_clean,page,i)
                i = i + 1
                
            if files_mode == False:
                df = pd.DataFrame([realestate])
                df_page = pd.concat([df_page,df],ignore_index=True)
              
        except:
            print(f'Error en página {page}: {url}') # + '\n' + str(prop_features_clean))
        
    return df_page

  def download_realestates(self,realestate,page,num):
    f = open('fotocasa/fotocasa_%s/realestate_%s_%s' % (page,page,num), 'w') # la W es para permisos de writing (escritura)
    f.write(realestate)
    f.close()
    
    return None

  def property_list(self, driver, city, page):
    # Recibo un número de página. Almaceno las urls de todas las propiedades de cada página de parrilla. Devuelvo una lista de urls.
    driver.get('https://www.fotocasa.es/es/comprar/viviendas/' + city.lower() + '-provincia/todas-las-zonas/l' + '/' + str(page))
        
    for scroll in range(40): # nos aseguramos que llega al final de la página
        ActionChains(driver).key_down(Keys.PAGE_DOWN).perform()
        time.sleep(0.1)
    
    property_url_list = []
    element = driver.find_elements(By.XPATH,'//a[contains(@class, "info-container")]') # or contains(@class, "carousel") or contains(@class, "slider")
    #print(f'Entra la página {page}. La longitud de element es {len(element)}')
    #print(f'Element de pag {page}: {element[0].get_attribute("href")}')

    [property_url_list.append(element[scroll].get_attribute('href')) for scroll in range(len(element))]
    
    return property_url_list

  def pages_to_scrape(self, driver, city):
    # Obtengo el número de páginas totales que debo recorrer. Devuelvo un entero.

    driver.get('https://www.fotocasa.es/es/comprar/viviendas/' + city.lower() + '-provincia/todas-las-zonas/l')
    
    page_selector = []
    while len(page_selector) < 1:
        html_txt = driver.page_source
        soup = BeautifulSoup(html_txt,'html.parser')
        page_selector = soup.findAll('li',attrs={'class':'sui-MoleculePagination-item'})
        ActionChains(driver).key_down(Keys.PAGE_DOWN).perform()
        time.sleep(0.1)
       
    n_pages = re.search('<span class="sui-AtomButton-inner">(.*)</span>',str(page_selector[-2])).group(1)
        
    return int(n_pages)

  def divide_pages(self, page_range):
        
    if page_range[1]-page_range[0]+1 >= 10:
        pages = []
        num_pages = list(range(page_range[0], page_range[1]))
        percentiles = [int(np.percentile(num_pages,x)) for x in range(0, 100, 20)]

        for loc in range(len(percentiles)):
            if percentiles[loc] == page_range[0]:
                start = page_range[0]
            else:
                start = percentiles[loc]+1

            if loc == len(percentiles)-1:
                end = page_range[1]
            else:
                end = percentiles[loc+1]

            pages.append([start, end+1])
            
        return pages
    else:
        return [[page_range[0],page_range[1]+1]]
    
    return None

  def scraping_loop(self, page_range, driver, city, download, files_mode):
    data_d = pd.DataFrame()
    
    for page in range(page_range[0], page_range[1]):
        properties_per_page = self.property_list(driver, city, page)
        property_data = self.parse_properties(driver, properties_per_page, page, download, files_mode)

        data_d = pd.concat([data_d,property_data], ignore_index=True)

        print(f'Página {str(page)} terminada.')
        
    return data_d

  def files_mode(self):
    path = os.getcwd()
    directory = 'fotocasa'
    df_files = pd.DataFrame()

    if os.path.exists(directory):
        os.chdir(directory)

        for page_number in os.listdir():
            if re.search('^fotocasa.*',page_number):
                page_directory = f'{page_number}'
            else:
                continue
            
            if os.path.exists(page_directory):
                os.chdir(page_directory) 
                
                for file in os.listdir():
                    if re.search('^realestate.*',file):
                        page = re.sub('_','',re.search('_(.*)_',file).group())
                        with open(f'{path}/{directory}/{page_directory}/{file}', 'r') as f:
                            prop_data = json.loads(f.read())
                            realestate = self.check_features(prop_data,page)

                            df = pd.DataFrame([realestate])
                            df_files = pd.concat([df_files,df],ignore_index=True)
            os.chdir(f'{path}/{directory}')
    else:
        print(f'No existe el directorio {directory}.')
    
    os.chdir(path)
    #print(os.getcwd())
                    
    return df_files

  def concurrent_scraping(self, range_driver_list, city, download, files_mode):
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        futures = []
        data = pd.DataFrame()
        
        for range_driver in range_driver_list:
            futures.append(executor.submit(self.scraping_loop, page_range = range_driver[0], driver = range_driver[1], city = city, download = download, files_mode = files_mode))

        for future in concurrent.futures.as_completed(futures):
            data = pd.concat([data,future.result()], ignore_index = True)
    
    return data

  def __init__(self, city = 'Madrid', page_range = [0,0], download = False, files_mode = False):
    '''
    Inicio de la clase con el scraping de Fotocasa para la ciudad indicada como parámetro.
    Por defecto: Madrid
    '''
    self.data = pd.DataFrame()
    directory = 'fotocasa'
    
    if files_mode == False or (files_mode == True and download == True):
        
        driver_1 = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=chrome_options)
        # Extraigo el máximo de páginas a scrapear de la parrilla de la ciudad
        total_pages = self.pages_to_scrape(driver_1, city)

        # Compruebo posibles errores en los rangos de páginas introducidos o el valor por defecto
        if page_range[0] == 0 and page_range[1] == 0:
            page_range[0] = 1
            page_range[1] = total_pages

        if page_range[1] > total_pages:
            page_range[1] = total_pages

        if page_range[0] > page_range[1]:
            aux = page_range[0]
            page_range[0] = page_range[1]
            page_range[1] = aux

        print('Pages to scrape: ' + str(page_range[1]-page_range[0]+1))
        
        if page_range[1] > 10:
            # Creo los webdrivers necesarios para la lectura concurrente
            #driver_1 = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=chrome_options)
            driver_2 = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=chrome_options)
            driver_3 = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=chrome_options)
            driver_4 = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=chrome_options)
            driver_5 = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=chrome_options)
            drivers = [driver_1, driver_2, driver_3, driver_4, driver_5]
        else:
            drivers = [driver_1] 
        
        # Divido el rango introducido en 5 grupos iguales
        page_range = self.divide_pages(page_range)
        print(page_range)
        range_driver_list = list(zip(page_range,drivers)) # asigno un driver a cada uno de los 5 rangos para no solapar lecturas del webdriver

    # Gestiono el modo lectura de archivos
    if files_mode == True:
        print('Modo lectura activado')
        if download == True:
            print('Modo descarga activado')
            if os.path.exists(directory):
                shutil.rmtree(directory) # si existe borro la carpeta y todo su contenido
            os.mkdir(directory)
            self.data = self.concurrent_scraping(range_driver_list, city, download, files_mode)
            self.data = self.files_mode()
        else:
            self.data = self.files_mode()
            
        print('Done!')
        self.data.to_csv('concurrent_scraping_from_files.csv',index=False)
        
    else:
        if download == True:
            print('Modo descarga activado')
            if os.path.exists(directory):
                shutil.rmtree(directory) # si existe borro la carpeta y todo su contenido
            os.mkdir(directory)

        self.data = self.concurrent_scraping(range_driver_list, city, download, files_mode)

        print('Done!')
        self.data.to_csv(f'concurrent_scraping_from_dataframe.csv',index=False)
    return None

CPU times: user 82 µs, sys: 25 µs, total: 107 µs
Wall time: 100 µs


## Modo de lectura y descarga de propiedades

In [296]:
%%time
ft_scraping_download = fotocasa_scraping(city = 'Madrid', download = True, files_mode = False)


Pages to scrape: 1012




[[1, 204], [204, 406], [406, 608], [608, 810], [810, 1013]]
Modo descarga activado
Página 1 terminada.
Página 608 terminada.
Página 204 terminada.
Página 406 terminada.
Página 810 terminada.
Página 2 terminada.
Página 609 terminada.
Página 205 terminada.
Página 407 terminada.
Página 811 terminada.
Página 3 terminada.
Página 610 terminada.
Página 206 terminada.
Página 408 terminada.
Página 812 terminada.
Página 4 terminada.
Página 611 terminada.
Página 207 terminada.
Página 409 terminada.
Página 813 terminada.
Página 5 terminada.
Página 612 terminada.
Página 208 terminada.
Página 410 terminada.
Página 814 terminada.
Página 6 terminada.
Página 613 terminada.
Página 209 terminada.
Página 411 terminada.
Página 815 terminada.
Página 7 terminada.
Página 614 terminada.
Página 412 terminada.
Página 210 terminada.
Página 816 terminada.
Página 8 terminada.
Página 615 terminada.
Página 211 terminada.
Página 413 terminada.
Página 817 terminada.
Página 9 terminada.
Página

Página 481 terminada.
Página 276 terminada.
Página 482 terminada.
Página 885 terminada.
Página 684 terminada.
Página 75 terminada.
Página 277 terminada.
Página 483 terminada.
Página 886 terminada.
Página 685 terminada.
Página 76 terminada.
Página 484 terminada.
Página 278 terminada.
Página 887 terminada.
Página 686 terminada.
Página 77 terminada.
Página 485 terminada.
Página 279 terminada.
Página 888 terminada.
Página 687 terminada.
Página 78 terminada.
Página 486 terminada.
Página 280 terminada.
Página 889 terminada.
Página 688 terminada.
Página 79 terminada.
Página 487 terminada.
Página 281 terminada.
Página 890 terminada.
Página 689 terminada.
Página 488 terminada.
Página 80 terminada.
Página 891 terminada.
Página 282 terminada.
Página 690 terminada.
Página 489 terminada.
Página 81 terminada.
Página 892 terminada.
Página 283 terminada.
Página 691 terminada.
Página 490 terminada.
Página 82 terminada.
Página 893 terminada.
Página 284 terminada.
Página 692 terminada.
Página 491 termina

Página 148 terminada.
Página 560 terminada.
Página 349 terminada.
Página 759 terminada.
Página 960 terminada.
Página 149 terminada.
Página 561 terminada.
Página 350 terminada.
Página 760 terminada.
Página 961 terminada.
Página 150 terminada.
Página 562 terminada.
Página 351 terminada.
Página 761 terminada.
Página 962 terminada.
Página 151 terminada.
Página 563 terminada.
Página 352 terminada.
Página 762 terminada.
Página 963 terminada.
Página 152 terminada.
Página 564 terminada.
Página 353 terminada.
Página 763 terminada.
Página 964 terminada.
Página 565 terminada.
Página 153 terminada.
Página 354 terminada.
Página 764 terminada.
Página 965 terminada.
Página 566 terminada.
Página 154 terminada.
Página 355 terminada.
Página 765 terminada.
Página 966 terminada.
Página 567 terminada.
Página 155 terminada.
Página 356 terminada.
Página 766 terminada.
Página 967 terminada.
Página 568 terminada.
Página 357 terminada.
Página 156 terminada.
Página 767 terminada.
Página 968 terminada.
Página 569

In [298]:
ft_scraping_download.data.shape

(30323, 31)

In [299]:
ft_scraping_download.data.head(10)

,origin,page,title,link,image_url,country,district,neighborhood,street,zipCode,...,surface,energyCertificate,hotWater,heating,conservationState,antiquity,floor,surfaceLand,otherFeatures,price
0,fotocasa,406,Piso en venta en Calle Aragón,https://www.fotocasa.es/es/comprar/vivienda/ar...,https://static.inmofactory.com/images/inmofact...,España,Zona el Caño,,Calle Aragón,28939,...,112,G,Gas Natural,Gas Natural,Muy bien,10 a 20 años,6,0,"{'2': 'Armarios', '11': 'Trastero', '12': 'Z. ...",224900
1,fotocasa,406,"Piso en venta en Calle Felix Rodriguez Fuente, 23",https://www.fotocasa.es/es/comprar/vivienda/al...,https://static.inmofactory.com/images/inmofact...,España,Algete - pueblo,,"Calle Felix Rodriguez Fuente, 23",28110,...,97,G,Gas Natural,Gas Natural,Muy bien,,8,0,{'131': 'Cocina Equipada'},167000
2,fotocasa,406,"Piso en venta en Calle General Ricardos, 138",https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.inmofactory.com/images/inmofact...,España,Carabanchel,San Isidro,"Calle General Ricardos, 138",28019,...,87,G,Gas Natural,,Muy bien,,6,0,"{'1': 'Aire acondicionado', '2': 'Armarios', '...",350000
3,fotocasa,406,"Piso en venta en Calle de Arturo Soria, 348",https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.inmofactory.com/images/inmofact...,España,Ciudad Lineal,Costillares,"Calle de Arturo Soria, 348",28033,...,179,G,Gas Natural,Gas Natural,Bien,20 a 30 años,11,0,"{'2': 'Armarios', '9': 'Parquet', '11': 'Trast...",643000
4,fotocasa,406,Piso en venta,https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.inmofactory.com/images/inmofact...,España,Tetuán,Cuatro Caminos - Azca,,28020,...,40,G,Gas Natural,Gas Natural,Muy bien,,5,0,"{'2': 'Armarios', '21': 'Electrodomésticos', '...",220000
5,fotocasa,406,Ático en venta en Calle de José Ortega y Gasset,https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.inmofactory.com/images/inmofact...,España,Barrio de Salamanca,Lista,Calle de José Ortega y Gasset,28006,...,91,G,Gas Natural,,Muy bien,,12,0,"{'1': 'Aire acondicionado', '2': 'Armarios', '...",575000
6,fotocasa,406,Piso en venta en Calle Barcelona,https://www.fotocasa.es/es/comprar/vivienda/mo...,https://static.inmofactory.com/images/inmofact...,España,Villafontana - Estoril I,,Calle Barcelona,28936,...,106,G,,,Bien,30 a 50 años,9,0,"{'1': 'Aire acondicionado', '3': 'Calefacción'...",240000
7,fotocasa,406,Casa o chalet en venta en Calle Teide,https://www.fotocasa.es/es/comprar/vivienda/lo...,https://static.fotocasa.es/images/anuncio/2022...,España,,,Calle Teide,28460,...,250,G,,,Bien,20 a 30 años,0,475,"{'2': 'Armarios', '3': 'Calefacción', '7': 'Ja...",500000
8,fotocasa,406,"Piso en venta en Avenida de Los Labradores, 7",https://www.fotocasa.es/es/comprar/vivienda/tr...,https://static.inmofactory.com/images/inmofact...,España,Estación - Centro,,"Avenida de Los Labradores, 7",28760,...,79,G,Gas Natural,Gas Natural,Bien,20 a 30 años,0,0,"{'1': 'Aire acondicionado', '2': 'Armarios', '...",308000
9,fotocasa,406,"Piso en venta en Calle Pío XII, 16",https://www.fotocasa.es/es/comprar/vivienda/co...,https://static.fotocasa.es/images/anuncio/0001...,España,Villalba Estación,,"Calle Pío XII, 16",28400,...,90,G,,,Bien,30 a 50 años,0,0,"{'10': 'Terraza', '32': 'Balcón'}",180000


In [300]:
ft_scraping_download.data['page'].nunique()

1011

In [301]:
ft_scraping_download.data.tail(10)

,origin,page,title,link,image_url,country,district,neighborhood,street,zipCode,...,surface,energyCertificate,hotWater,heating,conservationState,antiquity,floor,surfaceLand,otherFeatures,price
30313,fotocasa,203,Piso en venta,https://www.fotocasa.es/es/comprar/vivienda/ho...,https://static.inmofactory.com/images/inmofact...,España,,,,28240,...,90,G,,,,30 a 50 años,0,0,"{'2': 'Armarios', '3': 'Calefacción', '6': 'Gr...",355000
30314,fotocasa,203,Piso en venta en Rascón,https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.inmofactory.com/images/inmofact...,España,Carabanchel,San Isidro,Rascón,28019,...,40,G,,,,70 a 100 años,0,0,"{'6': 'Gres Cerámica', '84': 'Puerta Blindada'}",75000
30315,fotocasa,203,Dúplex en venta,https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.inmofactory.com/images/inmofact...,España,Centro,Universidad - Malasaña,,28004,...,48,G,,,,,0,0,"{'21': 'Electrodomésticos', '131': 'Cocina Equ...",190000
30316,fotocasa,203,Casa o chalet en venta,https://www.fotocasa.es/es/comprar/vivienda/bo...,https://static.inmofactory.com/images/inmofact...,España,Sector B,,,28660,...,245,G,,,,10 a 20 años,0,0,"{'1': 'Aire acondicionado', '2': 'Armarios', '...",750000
30317,fotocasa,203,Piso en venta,https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.inmofactory.com/images/inmofact...,España,Moncloa - Aravaca,Valdezarza,,28039,...,73,G,,,,10 a 20 años,0,0,{'11': 'Trastero'},203000
30318,fotocasa,203,Casa o chalet en venta,https://www.fotocasa.es/es/comprar/vivienda/co...,https://static.inmofactory.com/images/inmofact...,España,Ciudad 70,,,28822,...,133,G,,,,,0,0,"{'2': 'Armarios', '3': 'Calefacción', '10': 'T...",189900
30319,fotocasa,203,Piso en venta,https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.inmofactory.com/images/inmofact...,España,Hortaleza,Conde Orgaz - Piovera,,28043,...,87,G,,,,30 a 50 años,0,0,{},295000
30320,fotocasa,203,Casa o chalet en venta,https://www.fotocasa.es/es/comprar/vivienda/vi...,https://static.inmofactory.com/images/inmofact...,España,Los Hueros,,,28810,...,82,A,,,,,0,0,"{'3': 'Calefacción', '7': 'Jardín Privado', '1...",259000
30321,fotocasa,203,Casa o chalet en venta,https://www.fotocasa.es/es/comprar/vivienda/vi...,https://static.inmofactory.com/images/inmofact...,España,Villalbilla pueblo,,,28810,...,150,G,,,,,0,0,"{'2': 'Armarios', '3': 'Calefacción', '7': 'Ja...",263400
30322,fotocasa,203,^,^,^,^,^,^,^,^,...,^,^,^,^,^,^,^,^,^,0


In [302]:
ft_scraping_download.data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30323 entries, 0 to 30322
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   origin             30323 non-null  object
 1   page               30323 non-null  int64 
 2   title              30323 non-null  object
 3   link               30323 non-null  object
 4   image_url          30323 non-null  object
 5   country            30323 non-null  object
 6   district           30323 non-null  object
 7   neighborhood       30323 non-null  object
 8   street             30323 non-null  object
 9   zipCode            30323 non-null  object
 10  province           30323 non-null  object
 11  buildingType       30323 non-null  object
 12  clientAlias        30323 non-null  object
 13  latitude           30323 non-null  object
 14  longitude          30323 non-null  object
 15  isNewConstruction  30323 non-null  object
 16  rooms              30323 non-null  objec

In [303]:
ft_scraping_download.data.columns

Index(['origin', 'page', 'title', 'link', 'image_url', 'country', 'district',
       'neighborhood', 'street', 'zipCode', 'province', 'buildingType',
       'clientAlias', 'latitude', 'longitude', 'isNewConstruction', 'rooms',
       'bathrooms', 'parking', 'elevator', 'furnished', 'surface',
       'energyCertificate', 'hotWater', 'heating', 'conservationState',
       'antiquity', 'floor', 'surfaceLand', 'otherFeatures', 'price'],
      dtype='object')

## Modo solo de lectura de archivos

In [304]:
%%time
ft_scraping = fotocasa_scraping(download = False, files_mode = True)

Modo lectura activado
Done!
CPU times: user 2min 52s, sys: 3.9 s, total: 2min 56s
Wall time: 3min 3s


In [305]:
ft_scraping.data.shape

(30323, 31)

In [306]:
ft_scraping.data.head(10)

,origin,page,title,link,image_url,country,district,neighborhood,street,zipCode,...,surface,energyCertificate,hotWater,heating,conservationState,antiquity,floor,surfaceLand,otherFeatures,price
0,fotocasa,940,"Piso en venta en Calle de Fenelón, 11",https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.fotocasa.es/images/anuncio/2022...,España,San Blas,Canillejas,"Calle de Fenelón, 11",28022,...,95,G,,,Muy bien,20 a 30 años,0,0,"{'2': 'Armarios', '3': 'Calefacción', '10': 'T...",190000
1,fotocasa,940,"Piso en venta en Calle de la Reina Mercedes, 13",https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.fotocasa.es/images/anuncio/2022...,España,Tetuán,Cuatro Caminos - Azca,"Calle de la Reina Mercedes, 13",28020,...,140,G,,,Muy bien,50 a 70 años,0,0,"{'2': 'Armarios', '3': 'Calefacción', '9': 'Pa...",720000
2,fotocasa,940,Piso en venta en Calle Polvoranca,https://www.fotocasa.es/es/comprar/vivienda/hu...,https://static.inmofactory.com/images/inmofact...,España,,,Calle Polvoranca,28970,...,100,G,Gas Natural,Gas Natural,Muy bien,10 a 20 años,10,0,"{'1': 'Aire acondicionado', '2': 'Armarios', '...",200000
3,fotocasa,940,"Piso en venta en Calle Porto Lagos, 11",https://www.fotocasa.es/es/comprar/vivienda/al...,https://static.inmofactory.com/images/inmofact...,España,Parque Lisboa - La Paz,,"Calle Porto Lagos, 11",28924,...,104,G,,,Bien,30 a 50 años,8,0,"{'10': 'Terraza', '28': 'Serv. portería'}",260000
4,fotocasa,940,Piso en venta en Calle del Hierro,https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.fotocasa.es/images/anuncio/2022...,España,Arganzuela,Legazpi,Calle del Hierro,28045,...,118,G,,,Bien,10 a 20 años,0,0,"{'1': 'Aire acondicionado', '2': 'Armarios', '...",429999
5,fotocasa,940,"Piso en venta en Calle de Humanes, 4",https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.fotocasa.es/images/anuncio/2022...,España,Puente de Vallecas,Portazgo,"Calle de Humanes, 4",28038,...,81,G,,,Bien,30 a 50 años,0,0,"{'1': 'Aire acondicionado', '2': 'Armarios', '...",201000
6,fotocasa,940,"Ático en venta en Calle Isabel Méndez, 2",https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.fotocasa.es/images/anuncio/2022...,España,Puente de Vallecas,Numancia,"Calle Isabel Méndez, 2",28038,...,105,E,,,Casi nuevo,10 a 20 años,0,0,"{'2': 'Armarios', '3': 'Calefacción', '10': 'T...",299999
7,fotocasa,940,"Dúplex en venta en Calle Águilas, 1",https://www.fotocasa.es/es/comprar/vivienda/ar...,https://static.fotocasa.es/images/anuncio/2022...,España,Zona el Caño,,"Calle Águilas, 1",28939,...,214,G,,,Muy bien,5 a 10 años,0,0,"{'1': 'Aire acondicionado', '2': 'Armarios', '...",365000
8,fotocasa,940,"Piso en venta en Calle Juan Sánchez, 3",https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.fotocasa.es/images/anuncio/2022...,España,Fuencarral - El Pardo,Peñagrande,"Calle Juan Sánchez, 3",28035,...,87,G,,,Bien,10 a 20 años,0,0,"{'1': 'Aire acondicionado', '2': 'Armarios', '...",478000
9,fotocasa,940,Piso en venta en Calle de Gabriel Lobo,https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.fotocasa.es/images/anuncio/2022...,España,Chamartín,El Viso,Calle de Gabriel Lobo,28002,...,120,G,,,Muy bien,70 a 100 años,0,0,"{'1': 'Aire acondicionado', '2': 'Armarios', '...",760000


In [307]:
ft_scraping.data['page'].nunique()

1011

In [308]:
ft_scraping.data.tail(10)

,origin,page,title,link,image_url,country,district,neighborhood,street,zipCode,...,surface,energyCertificate,hotWater,heating,conservationState,antiquity,floor,surfaceLand,otherFeatures,price
30313,fotocasa,751,Piso en venta en Avenida Juan Pablo II,https://www.fotocasa.es/es/comprar/vivienda/pa...,https://static.fotocasa.es/images/anuncio/2022...,España,Miramadrid,,Avenida Juan Pablo II,28860,...,134,C,,,Casi nuevo,10 a 20 años,0,0,"{'1': 'Aire acondicionado', '2': 'Armarios', '...",475000
30314,fotocasa,751,Piso en venta en Calle de la Virgen de Los Rem...,https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.fotocasa.es/images/anuncio/2021...,España,Villa de Vallecas,Casco Histórico de Vallecas,"Calle de la Virgen de Los Remedios, 4",28031,...,89,F,,,,,0,0,"{'2': 'Armarios', '3': 'Calefacción', '9': 'Pa...",239000
30315,fotocasa,751,Piso en venta en Calle de Cartagena,https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.fotocasa.es/images/anuncio/2022...,España,Barrio de Salamanca,Guindalera,Calle de Cartagena,28028,...,144,G,,,Bien,50 a 70 años,0,0,"{'1': 'Aire acondicionado', '2': 'Armarios', '...",648000
30316,fotocasa,751,"Ático en venta en Calle Isabel Méndez, 2",https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.fotocasa.es/images/anuncio/2022...,España,Puente de Vallecas,Numancia,"Calle Isabel Méndez, 2",28038,...,105,E,,,Casi nuevo,10 a 20 años,0,0,"{'2': 'Armarios', '3': 'Calefacción', '10': 'T...",299999
30317,fotocasa,751,Piso en venta en Calle de Rodríguez San Pedro,https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.fotocasa.es/images/anuncio/2022...,España,Chamberí,Gaztambide,Calle de Rodríguez San Pedro,28015,...,152,G,,,A reformar,20 a 30 años,0,0,"{'1': 'Aire acondicionado', '3': 'Calefacción'...",595000
30318,fotocasa,751,"Piso en venta en Calle de Julio Palacios, 26",https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.fotocasa.es/images/anuncio/2022...,España,Fuencarral - El Pardo,La Paz,"Calle de Julio Palacios, 26",28029,...,161,G,,,Muy bien,20 a 30 años,0,0,"{'1': 'Aire acondicionado', '2': 'Armarios', '...",750000
30319,fotocasa,751,"Piso en venta en Avenida M-40, 3",https://www.fotocasa.es/es/comprar/vivienda/al...,https://static.fotocasa.es/images/anuncio/2022...,España,Campodón - Ventorro del Cano,,"Avenida M-40, 3",28925,...,120,G,,,Casi nuevo,5 a 10 años,0,0,"{'1': 'Aire acondicionado', '10': 'Terraza', '...",220000
30320,fotocasa,751,"Piso en venta en Calle Juan Sánchez, 3",https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.fotocasa.es/images/anuncio/2022...,España,Fuencarral - El Pardo,Peñagrande,"Calle Juan Sánchez, 3",28035,...,87,G,,,Bien,10 a 20 años,0,0,"{'1': 'Aire acondicionado', '2': 'Armarios', '...",478000
30321,fotocasa,751,"Dúplex en venta en Calle Águilas, 1",https://www.fotocasa.es/es/comprar/vivienda/ar...,https://static.fotocasa.es/images/anuncio/2022...,España,Zona el Caño,,"Calle Águilas, 1",28939,...,214,G,,,Muy bien,5 a 10 años,0,0,"{'1': 'Aire acondicionado', '2': 'Armarios', '...",365000
30322,fotocasa,751,Piso en venta en Calle de Federico Moreno Torr...,https://www.fotocasa.es/es/comprar/vivienda/ma...,https://static.fotocasa.es/images/anuncio/2022...,España,Retiro,Adelfas,"Calle de Federico Moreno Torroba, 11",28007,...,160,G,,,Bien,20 a 30 años,0,0,"{'1': 'Aire acondicionado', '2': 'Armarios', '...",720000


In [309]:
ft_scraping.data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30323 entries, 0 to 30322
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   origin             30323 non-null  object
 1   page               30323 non-null  object
 2   title              30323 non-null  object
 3   link               30323 non-null  object
 4   image_url          30323 non-null  object
 5   country            30323 non-null  object
 6   district           30323 non-null  object
 7   neighborhood       30323 non-null  object
 8   street             30323 non-null  object
 9   zipCode            30323 non-null  object
 10  province           30323 non-null  object
 11  buildingType       30323 non-null  object
 12  clientAlias        30323 non-null  object
 13  latitude           30323 non-null  object
 14  longitude          30323 non-null  object
 15  isNewConstruction  30323 non-null  object
 16  rooms              30323 non-null  objec

In [290]:
ft_scraping.data.columns

Index(['origin', 'page', 'title', 'link', 'image_url', 'country', 'district',
       'neighborhood', 'street', 'zipCode', 'province', 'buildingType',
       'clientAlias', 'latitude', 'longitude', 'isNewConstruction', 'rooms',
       'bathrooms', 'parking', 'elevator', 'furnished', 'surface',
       'energyCertificate', 'hotWater', 'heating', 'conservationState',
       'antiquity', 'floor', 'surfaceLand', 'otherFeatures', 'price'],
      dtype='object')